In [10]:
import pandas as pd


## life time MDD:
input_file1 = "out02_img_file_info_mdd_tms_response.csv"
out_file_prefix = 'life_time_mdd_'

## current MDD:
# input_file1 = "out02_img_file_info_current_mdd_tms_response.csv"
# out_file_prefix = 'current_mdd_'

#================================================================

# input_file2 = "out02_img_file_info_phq_tms_response.csv"
input_file3 = "out02_img_file_info_nthc_tms_response.csv"

subject_info = ["subject_id", "site"]
# subject_info = ["file_dir", "file_name", "subject_id"]

roi_cols = ["FSL_R_Amy_thr40", 
            "ROI_R_Amy_Com_MNI",
            "FIRST_R_amyg_small",
            "ROI_B_Amy_Com_MNI",
            "FSL_L_Amy_thr40",
            "FIRST_B_amyg_small",
            "FSL_B_Amy_thr40",
            "ROI_L_Amy_Com_MNI",
            "FIRST_L_amyg_small"]

data1 = pd.read_csv(input_file1)
# data2 = pd.read_csv(input_file2)
data3 = pd.read_csv(input_file3)

In [11]:

# data_all = pd.concat([data1[subject_info + roi_cols], 
#                       data2[subject_info + roi_cols], 
#                       data3[subject_info + roi_cols]],
#                      keys = ['mdd', 'phq', 'nthc'])

data = pd.concat([data1[subject_info + roi_cols], 
                      data3[subject_info + roi_cols]],
                     keys = ['mdd', 'nthc'])

data.reset_index(inplace = True, level = 0)
data.rename(columns = {'level_0': 'group'}, inplace = True)
data


,group,subject_id,site,FSL_R_Amy_thr40,ROI_R_Amy_Com_MNI,FIRST_R_amyg_small,ROI_B_Amy_Com_MNI,FSL_L_Amy_thr40,FIRST_B_amyg_small,FSL_B_Amy_thr40,ROI_L_Amy_Com_MNI,FIRST_L_amyg_small
0,mdd,3086,L_pMFG,-0.654121,-0.416947,-0.692253,-0.393486,-0.062504,-0.499196,-0.375819,-0.372316,-0.286281
1,mdd,3086,L_aMFG,0.039315,0.004902,0.050515,0.184007,0.246165,0.179610,0.136619,0.345625,0.321984
2,mdd,3086,R_M1,-0.265910,-0.446969,-0.227517,-0.688573,-0.787206,-0.533118,-0.511133,-0.906589,-0.870152
3,mdd,3045,L_pMFG,-0.291359,-0.258455,-0.263665,-0.442800,-0.610598,-0.424722,-0.441532,-0.609146,-0.602345
4,mdd,3045,L_aMFG,-0.360724,-0.531440,-0.254973,-0.262635,-0.343072,-0.258972,-0.352420,-0.020075,-0.263383
...,...,...,...,...,...,...,...,...,...,...,...,...
125,nthc,1108,R_M1,-0.362811,-0.329261,-0.354299,-0.328008,-0.504170,-0.392541,-0.429308,-0.326878,-0.434715
126,nthc,1068,R_M1,-0.127820,0.067308,-0.227763,0.381460,0.456773,0.141677,0.147178,0.664940,0.549116
127,nthc,1024,R_M1,-0.482517,-0.408466,-0.390370,-0.146074,0.029981,-0.196623,-0.241433,0.090700,0.017052
128,nthc,1053,R_M1,-0.349042,-0.357322,-0.406532,-0.389374,-0.635262,-0.508069,-0.483683,-0.418296,-0.620049


In [12]:
data[['group', 'site', 'subject_id']].groupby(['group', 'site']).count()

subject_id
group site              
mdd   L_aMFG           8
      L_pMFG           8
      R_M1             8
nthc  L_aMFG          42
      L_pMFG          45
      R_M1            43

In [5]:

pd.options.mode.chained_assignment = None # avoid the warnning message for assigning values to slices of df.
from scipy.stats import ttest_ind
import numpy as np

def format_table(report):
    report2 = report.copy()
    report2[report.gt(0)] = report2[report.gt(0)].astype(str).apply(lambda x : x.str[:5])
    report2[report.lt(0)] = report2[report.lt(0)].astype(str).apply(lambda x : x.str[:6])
    
    report_pvalue = report2.iloc[1::3,:]
    report_pvalue[report.iloc[1::3,:].le(0.05)] = \
    report_pvalue[report.iloc[1::3,:].le(0.05)].apply(lambda x : x.str[:5]).add('*')

    report_pvalue = report2.iloc[1::3,:]
    report_pvalue[report.iloc[1::3,:].le(0.01)] = \
    report_pvalue[report.iloc[1::3,:].le(0.01)].apply(lambda x : x.str[:5]).add('**')
        
    return report2

test_variables = [
                "FSL_R_Amy_thr40", 
                "FSL_L_Amy_thr40",
                "FSL_B_Amy_thr40",
                "FIRST_R_amyg_small",
                "FIRST_L_amyg_small",
                "FIRST_B_amyg_small",
                "ROI_R_Amy_Com_MNI",
                "ROI_L_Amy_Com_MNI",
                "ROI_B_Amy_Com_MNI",
               ]

tms_site = list(data['site'].unique())

row_index = pd.MultiIndex.from_tuples([(i , j) for i in tms_site for j in ['t value', 'p value', "Cohen's d"]])
result = pd.DataFrame(index = row_index, columns = test_variables)

# statistical tests for each roi and roi setting:
for var in test_variables:
    for tms in tms_site:
        hc = data.loc[(data['site']==tms) & (data['group']=='nthc') & (~data[var].isna()), var]
        mdd = data.loc[(data['site']==tms) & (data['group']=='mdd') & (~data[var].isna()), var]
        t, p = ttest_ind(hc, mdd)
        
        cohend = (np.mean(hc) - np.mean(mdd)) / (np.sqrt((np.std(hc) ** 2 + np.std(mdd) ** 2) / 2))
        
        result.loc[(tms, 't value'), var] = t
        result.loc[(tms, 'p value'), var] = p
        result.loc[(tms, "Cohen's d"), var] = cohend

result = format_table(result)
# save results separately for each roi setting:
# for var in test_variables:
#     result.loc[(tms, ['t value', 'p value', "Cohen's d"]), :].to_csv(out_file_prefix + 'response_' + var + '.csv')

result.to_csv(out_file_prefix + 'response_all.csv')
result.loc[(tms_site, ['p value', "Cohen's d"]),:]

FSL_R_Amy_thr40 FSL_L_Amy_thr40 FSL_B_Amy_thr40  \
L_pMFG p value             0.684           0.485           0.885   
       Cohen's d           0.215          -0.350          -0.075   
L_aMFG p value             0.371           0.635           0.470   
       Cohen's d          -0.402          -0.200          -0.305   
R_M1   p value             0.246           0.789           0.411   
       Cohen's d          -0.558          -0.115          -0.379   

                 FIRST_R_amyg_small FIRST_L_amyg_small FIRST_B_amyg_small  \
L_pMFG p value                0.652              0.454              0.874   
       Cohen's d              0.240             -0.374             -0.082   
L_aMFG p value                0.152              0.462              0.248   
       Cohen's d             -0.689             -0.325             -0.522   
R_M1   p value                0.194              0.837              0.376   
       Cohen's d             -0.615             -0.085             -0.402   

                 ROI_R_Amy_Com_MNI ROI_L_Amy_Com_MNI ROI_B_Amy_Com_MNI  
L_pMFG p value               0.880             0.413             0.680  
       Cohen's d             0.080            -0.381            -0.204  
L_aMFG p value               0.314             0.247             0.227  
       Cohen's d            -0.459            -0.535            -0.559  
R_M1   p value               0.083             0.758             0.256  
       Cohen's d            -0.780            -0.117            -0.477

In [4]:
tms_site

['L_pMFG', 'L_aMFG', 'R_M1']